In [10]:
import pandas as pd
import numpy as np
import os
import glob
from power_to_temperature import *
from test import *

In [11]:
freqs,vals,bwidth = bandwidth(50.,90.)
Freqs = np.linspace(1e-22,250,32769)
eta_nu = eta(Freqs)
mask = (Freqs>=50)&(Freqs<=91)
bwidth

P = lambda source: 10.0**((source - 30.) / 10.0) # dBm to Watts

In [19]:
folders = glob.glob('Data/*')
folders.sort()
i=-1
for subdirs, dirs, files in os.walk('Data'):
    dirs.sort()
    files.sort()
    K_jnc,measure = [],[]
    K_date,measure_date=[],[]

    for file in files:
        path = os.path.join(subdirs,file)
        date = file.split("_")[0]
        if  os.path.getsize(path)==0:
            print 'EMPTY FILE:',path
            continue
        elif file.endswith('short.dat'):
            dB_short = np.loadtxt(path,unpack=True)
            P_short = P(dB_short)
            #T_short = Res2Temp(dB_short,bwidth)
        elif file.endswith('50ohm.dat'):
            dB_50ohm = np.loadtxt(path,unpack=True)
            P_50ohm = P(dB_50ohm)
            #T_50ohm = Res2Temp(dB_50ohm,bwidth)
        elif file.endswith('noise.dat'):
            dB_noise = np.loadtxt(path,unpack=True)
            P_noise = P(dB_noise)
        elif file.endswith('antenna.dat'):
            dB_antenna = np.loadtxt(path,unpack=True)
            dB_clean = dB_antenna - dB_noise
            P_clean = P(dB_clean)
            #P_clean = P_antenna - P_noise
            #T_antenna = Radio_source_trans(dB_clean, Freqs, bwidth)
            Tamb = round(np.genfromtxt(path,comments='!',skip_header= 18,max_rows=1)[1]+273.15,2)
            Kjnc = Tamb/(dB_50ohm-dB_short)
            T_measure = Kjnc * (dB_clean/eta_nu - dB_short)
            
            print Tamb,T_measure[mask],(dB_50ohm-dB_short)[mask],(dB_clean/eta_nu - dB_short)[mask]
            K_jnc.append(Kjnc),measure.append(T_measure)
            K_date.append(date),measure_date.append(date)
    if i>=0 and i<len(folders) and K_jnc and measure:
        name = os.path.normpath(folders[i])
        name = name.split("/")[1]
        K_jnc = np.transpose(K_jnc)
        measure = np.transpose(measure)
        Kjnc_table = pd.DataFrame(K_jnc[mask], index = Freqs[mask], columns = K_date)
        Kjnc_table.to_hdf('Data_filter/K_jnc/'+name+'.hdf5','df')
        measure_table = pd.DataFrame(measure[mask], index = Freqs[mask], columns = measure_date)
        measure_table.to_hdf('Data_filter/Tmeas_jnc/'+name+'.hdf5','df')
    i+=1
    

298.43 [ 8022.86295818  7414.39111037  7115.23871701 ...,  6101.84240856
  6377.22849167  6393.10326326] [ 3.194862  3.489418  3.639984 ...,  4.07367   3.896523  3.886719] [ 85.88928726  86.69339476  86.78536034 ...,  83.29220375  83.26581608
  83.2630631 ]
298.43 [ 8036.26370897  7410.6488952   7100.0168824  ...,  6118.72153034
  6392.17936139  6404.43582141] [ 3.194862  3.489418  3.639984 ...,  4.07367   3.896523  3.886719] [ 86.03274988  86.6496386   86.59969792 ...,  83.52260944  83.46102571
  83.41065708]
298.43 [ 8033.0223658   7405.85391047  7106.89240099 ...,  6103.09678797
  6379.83207536  6400.54339329] [ 3.194862  3.489418  3.639984 ...,  4.07367   3.896523  3.886719] [ 85.99804946  86.59357283  86.68355939 ...,  83.30932645  83.2998104
  83.35996253]
298.43 [ 8029.65822103  7408.82869761  7106.92344804 ...,  6097.2750133
  6372.44291117  6397.04619683] [ 3.194862  3.489418  3.639984 ...,  4.07367   3.896523  3.886719] [ 85.96203439  86.62835578  86.68393808 ...,  83.2298572

KeyboardInterrupt: 

In [5]:
day14jnc = glob.glob('Data_filter/Tmeas_jnc/2013-06-14*')
dfs = []
for path in day14jnc:
    dfs.append(pd.read_hdf(path))
big_frame1 = pd.concat(dfs, axis = 1)
big_frame1.to_hdf('calibration/Day14_jnc.hdf5','df')

In [6]:
day15jnc = glob.glob('Data_filter/Tmeas_jnc/2013-06-15*')
dfs = []
for path in day15jnc:
    dfs.append(pd.read_hdf(path))
big_frame2 = pd.concat(dfs, axis = 1)
big_frame2.to_hdf('calibration/Day15_jnc.hdf5','df')

In [7]:
all_data = [big_frame1,big_frame2]
day14_15 = pd.concat(all_data,axis=1)
day14_15.to_hdf('calibration/Days14_15_jnc.hdf5','df')